In [1]:
//Australia having very distinct 8 climate zones,no single model will work for all australia 
// Even within a state  there are multiple climate zones(from alpine to high humid) 
//Different models need to be trained for different locations
//In this approach,separate models will be created for different locations and saved in the directory
//for the prediction particular location model will be loaded and prediction will be done
//Data used was weatherAUS_YN.csv(before imputing the  null values) as it was  observed  that 
//dropping null values are more effective than imputing (In our scenario imputing means blunting the features i.e.  generalising the data)
//Wind direction (categorical columns) not tested or used in the modelling as too much of encoding required and crashing the docker
//correlation matrix(wrangling file)& trial & Error methods were used to select the features)

Intitializing Scala interpreter ...

Spark Web UI available at http://64fd13194e1e:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591615976164)
SparkSession available as 'spark'


2020-06-08 11:32:49,920 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Import required libraries

In [2]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator,CrossValidatorModel}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
Logger.getLogger("org").setLevel(Level.ERROR)  

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator, CrossValidatorModel}
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator, VectorIndexer}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.log4j._


# Starting spark session

In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@577c468


# Importing the file

In [4]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("/home/weatherAUS-clean-FINAL.csv")

data: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [5]:
data.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

#  Selecting required fields with right data type

In [106]:
//based on correlation matrix following filelds were selected influencing the rainfall on the following day

In [107]:
val df1 = data.selectExpr("Date","Location",
                        "MaxTemp","Humidity3pm", 
                         "Pressure3pm",
                        "Temp3pm",
                        "Temp9am",
                         "RainToday_Sc",
                         "cast (Month as String) Month",
                         "RainTomorrow_Sc"
                      
                                                  )
                        

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


# Filtering and forming a particular location data

In [110]:
// Building a model to predict following day rainfal for one location

In [111]:
val LocData=df1.filter($"Location"==="Sydney")toDF()

LocData: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [112]:
// to see the no of records we have

In [113]:
LocData.count()

res47: Long = 3337


# Dropping the null values

In [114]:
val Loc =LocData.na.drop()

Loc: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [115]:
// checking the no of records after dropping null values

In [116]:
Loc.count()

res49: Long = 3337


# Setting the label column(Rain tomorrow)

In [117]:
val df2 = (Loc.select(Loc("RainTomorrow_Sc").as("label"), 
           $"Location",
           $"Date",
          $"MaxTemp", 
         $"Month",
         $"Raintoday_Sc",
          $"Humidity3pm",
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

df2: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


# Indexing the month column(categorical)-To capture the seasonal pattern

In [118]:
// Month  which is a categorical variable need  to be indexed and encoded to be used in logistic regression

In [119]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")


Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_72fd102314eb


# Encoding the MonIn( column

In [120]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))


Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_8d6af8db4055


# Setting the assembler

In [121]:
//setting the assembler with required fields for the regression model

In [122]:
 val assembler=(new VectorAssembler().setInputCols(Array(
                                                        "Raintoday_Sc",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_e9f6f4fb1c28


# Splitting the data into test and train sets

In [123]:
val Array(training,test)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


# Logistic regression

In [126]:
//Maximum iteration was set to 20,more iterations giving minor improvements in the accuracy but more runtime and docker failures
// as a trade off maximum iteration was set to 10

In [127]:
val lr=new LogisticRegression().setMaxIter(20)
       

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_5e9bb89cde84


# Setting the pipeline

In [128]:
val pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_09121463f2d1


# Cross validation hyper parameter tuning

In [129]:
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_5e9bb89cde84-regParam: 0.1
}, {
	logreg_5e9bb89cde84-regParam: 0.01
})


In [130]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)  

cv: org.apache.spark.ml.tuning.CrossValidator = cv_1112439e892a


In [131]:
val cvModel = cv.fit(training)

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_1112439e892a


# Transforming test data with model

In [132]:
val results =cvModel.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


# Model Evaluation

In [133]:
val predictions=results.select($"prediction",$"label").as[(Double,Double)].rdd

predictions: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[5876] at rdd at <console>:38


In [134]:
val metrics= new MulticlassMetrics(predictions)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@f4808f


In [135]:
println(metrics.confusionMatrix)

720.0  37.0   
125.0  133.0  


(1,1) no  pred no  rain
(2,2) yes pred yes rain
(1,2) no pred yes rain 
(2,1)  yes pred no rain 

In [136]:
metrics.accuracy

res55: Double = 0.8403940886699507


# Saving the model

In [137]:
cvModel.write.overwrite().save("/home/model_Sydney")

# Saving the pipeline

In [138]:
pipeline.write.overwrite().save("/home/pipeline_Sydney")

# Reloading the model

In [139]:
val modelLoaded=CrossValidatorModel.load("/home/model_Sydney")

modelLoaded: org.apache.spark.ml.tuning.CrossValidatorModel = cv_1112439e892a


# Using the model

In [140]:
// loaded model is applied on dataframe to see the prediction vs label

In [141]:
val dfu= modelLoaded.transform(df2)

dfu: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [142]:
df2.show() //Trying to check the actual Vs prediction for the sydney for last 10 years

+-----+--------+-------------------+-------+-----+------------+-----------+-----------+-------+-------+
|label|Location|               Date|MaxTemp|Month|Raintoday_Sc|Humidity3pm|Pressure3pm|Temp9am|Temp3pm|
+-----+--------+-------------------+-------+-----+------------+-----------+-----------+-------+-------+
|    0|  Sydney|2009-01-01 00:00:00|   34.7|    1|           0|       22.0|     1000.7|   22.0|   34.3|
|    0|  Sydney|2009-01-02 00:00:00|   22.7|    1|           0|       57.0|     1013.8|   20.1|   20.8|
|    0|  Sydney|2009-01-03 00:00:00|   23.0|    1|           0|       49.0|     1019.4|   18.8|   21.4|
|    0|  Sydney|2009-01-04 00:00:00|   24.6|    1|           0|       58.0|     1015.2|   23.0|   24.1|
|    0|  Sydney|2009-01-05 00:00:00|   27.9|    1|           0|       60.0|     1008.9|   23.4|   26.2|
|    0|  Sydney|2009-01-06 00:00:00|   28.3|    1|           0|       64.0|     1008.0|   23.1|   25.8|
|    0|  Sydney|2009-01-07 00:00:00|   33.1|    1|           0| 

In [143]:
val testev=dfu.select("Location","Date","RawPrediction","probability","label","prediction").toDF()

testev: org.apache.spark.sql.DataFrame = [Location: string, Date: timestamp ... 4 more fields]


In [144]:
testev.show()

+--------+-------------------+--------------------+--------------------+-----+----------+
|Location|               Date|       RawPrediction|         probability|label|prediction|
+--------+-------------------+--------------------+--------------------+-----+----------+
|  Sydney|2009-01-01 00:00:00|[4.57331538904891...|[0.98978181333271...|    0|       0.0|
|  Sydney|2009-01-02 00:00:00|[1.29746752156461...|[0.78540846255106...|    0|       0.0|
|  Sydney|2009-01-03 00:00:00|[1.87603926645339...|[0.86715552568127...|    0|       0.0|
|  Sydney|2009-01-04 00:00:00|[1.46081926110891...|[0.81165794665687...|    0|       0.0|
|  Sydney|2009-01-05 00:00:00|[1.50137464719798...|[0.81777941044916...|    0|       0.0|
|  Sydney|2009-01-06 00:00:00|[1.22247032067356...|[0.77249798777921...|    0|       0.0|
|  Sydney|2009-01-07 00:00:00|[1.13528386767648...|[0.75681269949008...|    0|       0.0|
|  Sydney|2009-01-08 00:00:00|[0.26646608225673...|[0.56622512846416...|    0|       0.0|
|  Sydney|

In [ ]:
// future ideas to expand
// creating models to different climate zones
// trying to predict next couple of days rainfall( e.g. 3 days) with the available data
// Trying deep learning neural networks for better models
// Estimate the amount of rainfall